### Traditional Model Solution

In [1]:
import math
import numpy
from gurobipy import *
import os

p = 0.8
sigma=math.sqrt((1-p)/29/29+p/300-((1-p)/29+p/30)*((1-p)/29+p/30))
mu_model = (1-p)/29+p/30
c = 0
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]

S_MAX = 101
N = 22
Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)

for i in range (N): #re-scaling
    b_List[i] = b_List[i] *100 / 100.0
    h_List[i] = h_List[i] / 100.0
            
def optimal_solution (k):
    F_S_S = []
    solution = []
    for S_in in range (0,S_MAX):
        temp = []
        for S_out in range (0,S_MAX):
            SubCost = []
            for j in range (N):
                if ((S_in+L_List[j]-S_out) >= 0):
                    SubCost.append(h_List[j]*k*sigma*math.sqrt(S_in+L_List[j]-S_out))
                else: 
                    SubCost.append(5000)
            temp.append(SubCost)
        F_S_S.append(temp)

    F_S_S = numpy.array(F_S_S)
    print("F_S_S generated!")

    #Solve MILP
    try:
        m = Model("Algo-Trad")
        # Create variables
        chi = m.addVars(N, S_MAX, S_MAX, vtype=GRB.BINARY, name="chi")
        # Set objective
        obj = 0
        for j in range (N):
            for mu in range (S_MAX):
                for nu in range (S_MAX):
                    obj += F_S_S[mu,nu,j]*chi[j,mu,nu]
        m.setObjective(obj, GRB.MINIMIZE)
        # Add constraint
        for j in range (N):
            m.addConstr(quicksum(chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == 1)
            if (S_out_customer[j] < 1e20):
                m.addConstr(quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_out_customer[j])
            if (S_in_supplier[j] < 1e20):
                m.addConstr(quicksum(mu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_in_supplier[j])
        for i in range (len(Non_Start)):
            for j in Structure[Non_Start[i]]:
                m.addConstr(quicksum(mu*chi[Non_Start[i],mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) >= quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)))

        m.optimize()

    except GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')

    for j in range (N):
        for mu in range (S_MAX):
            for nu in range (S_MAX):
                if(chi[j,mu,nu].X == 1):
                    solution.append([F_S_S[mu,nu,j],mu,nu]) #obj,S_in,S_out
    
    return solution.copy()

Trad_Binomial = open("Trad-Binomial-Simulation.txt", mode='w')
Trad_Poisson = open("Trad-Poisson-Simulation.txt", mode='w')
Trad_Solution = open("Trad-Solution.txt", mode='w')
for k in [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,
         2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,
          3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,
         4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0]:
    opt_temp = optimal_solution(k)
    opt_temp = numpy.array(opt_temp)
    S_in_List = opt_temp[:,1]
    S_out_List = opt_temp[:,2]
    S_in_List = S_in_List.tolist()
    S_out_List = S_out_List.tolist()
    W_List = S_in_List
    B_List = []
    for i in range (N):
        B_List.append(round(mu_model*(S_in_List[i]+L_List[i]-S_out_List[i])+k*sigma*math.sqrt(S_in_List[i]+L_List[i]-S_out_List[i])))        
        
    Trad_Solution.write(str(k)+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_in_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_out_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(B_List[i])+'\t')
    Trad_Solution.write('\n') 
    #simulation-binomial
    for w in range (100):
        numpy.random.seed(w+10)
        Z = numpy.random.binomial(1,p,500).tolist() #Z-29
        demand = [] #real-time demand 0 or 1
        for z in Z:
            for tempintdemand in range (28):
                demand.append(0)
            if (z == 0):
                demand.append(1)
            else:
                demand.append(0)
                demand.append(1)
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Binomial.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Binomial.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Binomial.write(str(backordertimes[i])+'\t')
        Trad_Binomial.write('\n')
    print(k, "Simulation-Binomial Ended!")
    #simulation-poisson
    for w in range (100):
        numpy.random.seed(w+10)
        demand = numpy.random.poisson(mu_model,int(((1-p)*29+p*30)*500)).tolist()
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Poisson.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Poisson.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Poisson.write(str(backordertimes[i])+'\t')
        Trad_Poisson.write('\n')
    print(k, "Simulation-Poisson Ended!")
Trad_Binomial.close() #every row: k,w,cost of stages 0~21, backorder times of stages 0~21
Trad_Poisson.close()
Trad_Solution.close() #every row: k, inbound service time of stages 0~21, outbound service time of stages 0~21, base stock of stages 0~21

F_S_S generated!

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-07-13
Using license file /Users/estelle/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x1291964d
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 133.2846680
Presolve removed 15 rows and 199464 columns (presolve time = 7s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 7.04s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 109.0912154
Variable types: 0 continuous, 23908 integer (23908

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 159.9416016
Presolve removed 15 rows and 200525 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.13s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 130.9094585
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      5s
      85    3.7379690e+01   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 3.737969e+01, 85 iterations, 0.02 seconds
Total elapsed time = 5.39s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   37.37969    0   30  130.90946   37.37969  71.4%     -    5s
H    0     0                      89.5167566   37.37969

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x71fdf235
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 186.5985352
Presolve removed 15 rows and 199464 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.29s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 152.7277015
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      6s
      85    4.3609639e+01   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 4.360964e+01, 85 iteration

Total elapsed time = 5.34s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   49.83959    0   30  174.54594   49.83959  71.4%     -    5s
H    0     0                     119.3556754   49.83959  58.2%     -    5s
H    0     0                      95.0278147   49.83959  47.6%     -    5s
H    0     0                      93.7055776   49.83959  46.8%     -    5s
     0     0   58.03630    0   34   93.70558   58.03630  38.1%     -    5s
H    0     0                      87.8842269   58.03630  34.0%     -    5s
     0     0   59.81613    0   33   87.88423   59.81613  31.9%     -    5s
     0     0   69.44593    0   57   87.88423   69.44593  21.0%     -    5s
     0     0   69.44593    0   31   87.88423   69.44593  21.0%     -    5s
     0     0   69.44593    0   49   87.88423   69.44593  21.0%     -    5s
H    0     0                      87.8614061   69.44593  21.0%     -   

     0     0   65.29083    0   34  105.41877   65.29083  38.1%     -    5s
H    0     0                      98.8697552   65.29083  34.0%     -    5s
     0     0   67.29314    0   33   98.86976   67.29314  31.9%     -    5s
     0     0   78.12667    0   57   98.86976   78.12667  21.0%     -    5s
     0     0   78.12667    0   31   98.86976   78.12667  21.0%     -    5s
     0     0   78.12667    0   49   98.86976   78.12667  21.0%     -    6s
H    0     0                      98.8440818   78.12667  21.0%     -    6s
     0     0   79.34128    0   55   98.84408   79.34128  19.7%     -    6s
     0     0   79.59913    0   53   98.84408   79.59913  19.5%     -    6s
     0     0   79.59913    0   54   98.84408   79.59913  19.5%     -    6s
     0     0   85.16473    0   39   98.84408   85.16473  13.8%     -    6s
     0     0   85.35326    0   39   98.84408   85.35326  13.6%     -    6s
     0     0   85.36623    0   40   98.84408   85.36623  13.6%     -    6s
     0     0   85.82746  

...

     0     0  199.18786    0   40  230.63619  199.18786  13.6%     -    5s
     0     0  200.26407    0   53  230.63619  200.26407  13.2%     -    5s
H    0     0                     204.0773794  200.26407  1.87%     -    6s
     0     0  200.64460    0   45  204.07738  200.64460  1.68%     -    6s
     0     0  200.76203    0   43  204.07738  200.76203  1.62%     -    6s
     0     0  201.29168    0   43  204.07738  201.29168  1.37%     -    6s
H    0     0                     203.6959914  201.29168  1.18%     -    6s
     0     0  201.29168    0   26  203.69599  201.29168  1.18%     -    6s
     0     0  202.54657    0   31  203.69599  202.54657  0.56%     -    6s
     0     0  203.38424    0    2  203.69599  203.38424  0.15%     -    6s
     0     0     cutoff    0       203.69599  203.69599  0.00%     -    6s

Explored 1 nodes (542 simplex iterations) in 6.14 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 203.696 204.077 230.636 ... 559.796

Optimal solut

     0     0  212.56938    0   51  213.75749  212.56938  0.56%     -    6s
     0     0  212.95550    0    8  213.75749  212.95550  0.38%     -    6s
H    0     0                     213.3958006  212.95550  0.21%     -    6s
     0     0  213.06921    0    2  213.39580  213.06921  0.15%     -    6s

Explored 1 nodes (653 simplex iterations) in 6.17 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 213.396 213.757 214.737 ... 480.001

Optimal solution found (tolerance 1.00e-04)
Best objective 2.133958005571e+02, best bound 2.133958005571e+02, gap 0.0000%
4.4 Simulation-Binomial Ended!
4.4 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xcbe424b0
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00

Best objective 2.230956096734e+02, best bound 2.230956096734e+02, gap 0.0000%
4.6 Simulation-Binomial Ended!
4.6 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x2e3efee0
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 626.4379396
Presolve removed 15 rows and 199464 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.18s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 512.7287123
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal

     0     0  232.33230    0    2  232.79542  232.33230  0.20%     -    6s

Cutting planes:
  Gomory: 3
  Clique: 2
  StrongCG: 1
  GUB cover: 7
  Zero half: 2
  RLT: 5

Explored 1 nodes (648 simplex iterations) in 6.15 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 232.795 233.19 234.258 ... 523.638

Optimal solution found (tolerance 1.00e-04)
Best objective 2.327954187896e+02, best bound 2.327954187896e+02, gap 0.0000%
4.8 Simulation-Binomial Ended!
4.8 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x45791dbd
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [6e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solut

5.0 Simulation-Poisson Ended!


In [2]:
import os
import numpy

N = 22

File = open("Trad-Solution.txt", mode='r')
temp = []
while True:
    Line = File.readline()
    if Line:
        Line = Line[:-2]
        Line = list(map(float,Line.split('\t')))
        Line = Line[1:]
        temp.append(Line)
    else:
        break
File.close()
print("inbound service time")
print(numpy.average(temp, axis=0)[0:N])
print(numpy.max(temp, axis=0)[0:N])
print(numpy.min(temp, axis=0)[0:N])
print("outbound service time")
print(numpy.average(temp, axis=0)[N:(2*N)])
print(numpy.max(temp, axis=0)[N:(2*N)])
print(numpy.min(temp, axis=0)[N:(2*N)])
print("base stock")
print(numpy.average(temp, axis=0)[(2*N):(3*N)])
print(numpy.max(temp, axis=0)[(2*N):(3*N)])
print(numpy.min(temp, axis=0)[(2*N):(3*N)])

inbound service time
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
outbound service time
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
base stock
[0.17073171 0.         1.         1.09756098 1.09756098 1.
 1.         1.         1.90243902 1.09756098 0.         1.
 1.         0.         0.         0.         1.2195122  2.3902439
 1.09756098 1.         0.53658537 0.        ]
[1. 0. 1. 2. 2. 1. 1. 1. 2. 2. 0. 1. 1. 0. 0. 0. 2. 3. 2. 1. 1. 0.]
[0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 2. 1. 1. 0. 0.]
